In [1]:
# %% [markdown]
# # SAM Training Notebook
# Converted from Python script for interactive execution in Jupyter.
# 
# This notebook preserves the original script logic and allows step-by-step debugging.

# %% [code]
import os
import time
import random
import argparse

import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
from matplotlib import cm

import lightning as L
from lightning.fabric.loggers import TensorBoardLogger
from lightning.fabric.fabric import _FabricOptimizer

from box import Box
from datasets import call_load_dataset
from utils.model import Model
from utils.eval_utils import AverageMeter, validate, get_prompts, calc_iou
from utils.tools import copy_model, create_csv, reduce_instances
from utils.utils import *

/home/tic/miniconda3/envs/cvpr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# %% [markdown]
# ## Training Function

# %% [code]
def train_sam(cfg: Box,
              fabric: L.Fabric,
              model: Model,
              optimizer: _FabricOptimizer,
              scheduler: _FabricOptimizer,
              train_dataloader: DataLoader,
              val_dataloader: DataLoader,
              target_pts):
    model.eval()

    save_dir = "entropy_sorted"
    os.makedirs(save_dir, exist_ok=True)

    collected = []
    with torch.no_grad():
        for i, batch in enumerate(tqdm(target_pts, desc='Computing per-sample entropy', ncols=100)):
            imgs, boxes, masks, img_paths = batch
            prompts = get_prompts(cfg, boxes, masks)
            embeds, masks_pred, _, _ = model(imgs, prompts)

            batch_size = imgs.shape[0]
            for b in range(batch_size):
                img_np = (imgs[b].permute(1,2,0).cpu().numpy() * 255).astype(np.uint8)
                p_b = masks_pred[b].clamp(1e-6, 1 - 1e-6)
                if p_b.ndim == 2:
                    p_b = p_b.unsqueeze(0)
                gt_b = masks[b]
                if gt_b.ndim == 2:
                    gt_b = gt_b.unsqueeze(0)

                entropy_scalar = 0
                num_inst = p_b.shape[0]
                for j in range(num_inst):
                    p_inst = p_b[j]
                    entropy_map_inst = - (p_inst * torch.log(p_inst) + (1 - p_inst) * torch.log(1 - p_inst))
                    entropy_scalar += float(entropy_map_inst.mean().cpu().item())

                entropy_scalar /= num_inst
                render = {
                    'real': img_np,
                    'prompt': prompts
                }
                img_path = img_paths[b] if isinstance(img_paths, (list, tuple)) else img_paths
                collected.append((entropy_scalar, img_path, render))

    collected.sort(key=lambda x: x[0], reverse=True)

    return collected

    

# %% [markdown]
# ## Optimizer and Scheduler Configuration

# %% [code]
def configure_opt(cfg: Box, model: Model):
    def lr_lambda(step):
        if step < cfg.opt.warmup_steps:
            return step / cfg.opt.warmup_steps
        elif step < cfg.opt.steps[0]:
            return 1.0
        elif step < cfg.opt.steps[1]:
            return 1 / cfg.opt.decay_factor
        else:
            return 1 / (cfg.opt.decay_factor**2)

    optimizer = torch.optim.Adam(model.model.parameters(),
                                 lr=cfg.opt.learning_rate,
                                 weight_decay=cfg.opt.weight_decay)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
    return optimizer, scheduler





In [3]:
# %% [markdown]
# ## Configuration Loading and Launch

# %% [code]
# Example: set arguments manually here
# Replace with your config module path, e.g. "configs.default_config"
import importlib

CFG_MODULE = "configs.config_nwpu_"
cfg_module = importlib.import_module(CFG_MODULE)
cfg = cfg_module.cfg

# Manually merge updates if needed
cfg.out_dir = "./outputs"
cfg.resume = False

torch.cuda.empty_cache()
torch.set_float32_matmul_precision('high')

# main(cfg)

In [4]:
gpu_ids = [str(i) for i in range(torch.cuda.device_count())]
fabric = L.Fabric(accelerator="auto",
                  devices=len(gpu_ids),
                  strategy="auto",
                  loggers=[TensorBoardLogger(cfg.out_dir)])
fabric.launch()
fabric.seed_everything(1337 + fabric.global_rank)

if fabric.global_rank == 0:
    os.makedirs(os.path.join(cfg.out_dir, "save"), exist_ok=True)
    create_csv(os.path.join(cfg.out_dir, "metrics.csv"), csv_head=cfg.csv_keys)

with fabric.device:
    model = Model(cfg)
    model.setup()

load_datasets = call_load_dataset(cfg)
train_data, val_data, pt_data = load_datasets(cfg, img_size=1024, return_pt=True)
train_data = fabric._setup_dataloader(train_data)
val_data = fabric._setup_dataloader(val_data)
pt_data = fabric._setup_dataloader(pt_data)

optimizer, scheduler = configure_opt(cfg, model)
model, optimizer = fabric.setup(model, optimizer)

if cfg.resume and cfg.model.ckpt is not None:
    full_checkpoint = fabric.load(cfg.model.ckpt)
    model.load_state_dict(full_checkpoint["model"])
    optimizer.load_state_dict(full_checkpoint["optimizer"])

collected = train_sam(cfg, fabric, model, optimizer, scheduler, train_data, val_data, pt_data)



Global seed set to 1337


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


Computing per-sample entropy: 100%|███████████████████████████████| 520/520 [15:20<00:00,  1.77s/it]


In [5]:
# _, _, = validate(fabric, cfg, model, val_data, name=cfg.name, epoch=0)

In [9]:
def prompt_calibration(cfg, entrop_map, prompts, point_status):
    point_list = []
    point_labels_list = []
    num_points = cfg.num_points

    for m in range(len(entrop_map)):
        point_coords = prompts[0][0][m][:].unsqueeze(0)
        point_coords_lab = prompts[0][1][m][:].unsqueeze(0)

        # Find high-entropy location
        max_idx = torch.argmax(entrop_map[m])
        y = max_idx // entrop_map[m].shape[1]
        x = max_idx % entrop_map[m].shape[1]
        neg_point_coords = torch.tensor([[x.item(), y.item()]], device=point_coords.device).unsqueeze(0)


        # Combine positive and negative points
        point_coords_all = torch.cat((point_coords, neg_point_coords), dim=1)
        
        # Append a new label (1) to the label tensor
        point_labels_all = torch.cat(
            (point_coords_lab, torch.tensor([[point_status]], device=point_coords.device, dtype=point_coords_lab.dtype)),
            dim=1
        )
        
        point_list.append(point_coords_all)
        point_labels_list.append(point_labels_all)



    point_ = torch.cat(point_list).squeeze(1)
    point_labels_ = torch.cat(point_labels_list)
    new_prompts = [(point_, point_labels_)]
    return new_prompts


In [10]:
def entropy_map_calculate(p):
    entropy_map = - (p * torch.log(p) + (1 - p) * torch.log(1 - p))
    entropy_map = entropy_map.max(dim=0)[0]

    return entropy_map

In [11]:
def process_forward(img_tensor, prompt):
    with torch.no_grad():
        _, masks_pred, _, _ = model(img_tensor, prompt)
    entropy_maps = []
    pred_ins = []
    for i, mask_p in enumerate( masks_pred[0]):

        p = mask_p.clamp(1e-6, 1 - 1e-6)
        if p.ndim == 2:
            p = p.unsqueeze(0)

        entropy_map = entropy_map_calculate(p)
        entropy_maps.append(entropy_map)
        pred_ins.append(p)

    return entropy_maps, pred_ins
        
        
        
    

In [17]:
save_dir="entropy_sorted"
for rank, (entropy_scalar, img_path, render) in enumerate(reversed(collected), start=1):
    img_tensor = torch.from_numpy(render['real']).permute(2,0,1).float() / 255.0
    img_tensor = img_tensor.unsqueeze(0).to(fabric.device)

    prompt_main = render['prompt']

    entropy_maps, preds = process_forward(img_tensor, prompt_main)

    

    prompt_1 = prompt_calibration(cfg, entropy_maps, prompt_main, 1)
    entropy_maps_1, preds_1 = process_forward(img_tensor, prompt_1)
    entr_means_pos = [ent.mean().item() for ent in entropy_maps_1]

    # Compute relative entropy = mean entropy / (number of 1s in prediction)
    rel_entr_pos = []
    for ent, pred in zip(entropy_maps_1, preds_1):
        num_ones = (pred >0.5 ).sum().item()
        rel_entr = ent.mean().item() / num_ones if num_ones > 0 else float('inf')
        rel_entr_pos.append(rel_entr)


    prompt_2 = prompt_calibration(cfg, entropy_maps, prompt_main, 0)
    entropy_maps_2, preds_2 = process_forward(img_tensor, prompt_2)
    entr_means_neg = [ent.mean().item() for ent in entropy_maps_2]

    rel_entr_neg = []
    for ent, pred in zip(entropy_maps_2, preds_2):
        num_ones = (pred == 0.5).sum().item()
        rel_entr = ent.mean().item() / num_ones if num_ones > 0 else float('inf')
        rel_entr_neg.append(rel_entr)

    pred_final = []
    
    
    for i in range(len(preds_1)):
        if rel_entr_pos[i] < rel_entr_neg[i]:
            pred_final.append(preds_1[i])
        else:
            pred_final.append(preds_2[i])
        
   
    # for i,p in  enumerate(pred_final):
    #     pred_mask = p.max(dim=0)[0]
    #     pred_bin = (pred_mask > 0.5).to(torch.uint8).cpu().numpy() * 255


    
    #     base = os.path.splitext(os.path.basename(img_path))[0]
    #     suffix = f"{rank:05d}_{base}"
        
    #     Image.fromarray(pred_bin).save(os.path.join(save_dir, f"{suffix}{i}_pred.jpg"))
    #     # Image.fromarray(entropy_color).save(os.path.join(save_dir, f"{suffix}{i}_en.jpg"))

    # Combine all pseudo-label predictions into one final mask
    pred_stack = torch.stack([
        (p.max(dim=0)[0] if p.ndim == 3 else p) for p in pred_final
    ])
    final_mask = pred_stack.mean(dim=0)  # or .max(dim=0)[0] for union
    # Convert to binary pseudo label
    final_bin = (final_mask > 0.06).to(torch.uint8).cpu().numpy() * 255

    # Save combined pseudo-label
    base = os.path.splitext(os.path.basename(img_path))[0]
    suffix = f"{rank:05d}_{base}"
    Image.fromarray(final_bin).save(os.path.join(save_dir, f"{suffix}_pseudo_combined.jpg"))


    # Combine all pseudo-label predictions into one final mask
    pred_first = torch.stack([
        (p.max(dim=0)[0] if p.ndim == 3 else p) for p in preds
    ])
    final_mask = pred_first.mean(dim=0)  # or .max(dim=0)[0] for union
    # Convert to binary pseudo label
    final_bin_p = (final_mask > 0.06).to(torch.uint8).cpu().numpy() * 255

    # Save combined pseudo-label
    base = os.path.splitext(os.path.basename(img_path))[0]
    suffix = f"{rank:05d}_{base}"
    Image.fromarray(final_bin_p).save(os.path.join(save_dir, f"{suffix}_pred_combined.jpg"))
    

    
    Image.fromarray(render['real']).save(os.path.join(save_dir, f"{suffix}.jpg"))


    if rank>400:
        break

    

    

OutOfMemoryError: CUDA out of memory. Tried to allocate 768.00 MiB. GPU 0 has a total capacity of 5.68 GiB of which 204.12 MiB is free. Including non-PyTorch memory, this process has 5.46 GiB memory in use. Of the allocated memory 4.21 GiB is allocated by PyTorch, and 1.12 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
save_dir="entropy_sorted"

for rank, (entropy_scalar, img_path, render) in enumerate(collected, start=1):
    img_tensor = torch.from_numpy(render['real']).permute(2,0,1).float() / 255.0
    img_tensor = img_tensor.unsqueeze(0).to(fabric.device)

    prompt_b = render['prompt']


    with torch.no_grad():
        _, masks_pred, _, _ = model(img_tensor, prompt_b)



    entropy_maps = []
    for i, mask_pred in enumerate( masks_pred[0]):
  

        p = mask_pred.clamp(1e-6, 1 - 1e-6)
        if p.ndim == 2:
            p = p.unsqueeze(0)
    
        entropy_map = - (p * torch.log(p) + (1 - p) * torch.log(1 - p))
        entropy_map = entropy_map.max(dim=0)[0]
        pred_mask = p.max(dim=0)[0]
        pred_bin = (pred_mask > 0.5).to(torch.uint8).cpu().numpy() * 255

        entropy_maps.append(entropy_map)
    
        entropy_np = entropy_map.cpu().numpy()
        entropy_norm = (entropy_np - entropy_np.min()) / (entropy_np.max() - entropy_np.min() + 1e-6)
        cmap = cm.get_cmap("viridis")
        entropy_color = (cmap(entropy_norm)[:, :, :3] * 255).astype(np.uint8)

    
        base = os.path.splitext(os.path.basename(img_path))[0]
        suffix = f"{rank:05d}_{base}"
        Image.fromarray(render['real']).save(os.path.join(save_dir, f"{suffix}{i}.jpg"))
        Image.fromarray(pred_bin).save(os.path.join(save_dir, f"{suffix}{i}_pred.jpg"))
        Image.fromarray(entropy_color).save(os.path.join(save_dir, f"{suffix}{i}_en.jpg"))
    new_prom = neg_prompt_calibration(cfg, entropy_maps, prompt_b) 


    # with torch.no_grad():
    #     _, masks_pred, _, _ = model(img_tensor, new_prom)

    # entropy_maps = []
    # for i, mask_pred in enumerate( masks_pred[0]):
  

    #     p = mask_pred.clamp(1e-6, 1 - 1e-6)
    #     if p.ndim == 2:
    #         p = p.unsqueeze(0)
    
    #     entropy_map = - (p * torch.log(p) + (1 - p) * torch.log(1 - p))
    #     entropy_map = entropy_map.max(dim=0)[0]
    #     pred_mask = p.max(dim=0)[0]
    #     pred_bin = (pred_mask > 0.5).to(torch.uint8).cpu().numpy() * 255

    #     entropy_maps.append(entropy_map)
    
    #     entropy_np = entropy_map.cpu().numpy()
    #     entropy_norm = (entropy_np - entropy_np.min()) / (entropy_np.max() - entropy_np.min() + 1e-6)
    #     cmap = cm.get_cmap("viridis")
    #     entropy_color = (cmap(entropy_norm)[:, :, :3] * 255).astype(np.uint8)

    
    #     base = os.path.splitext(os.path.basename(img_path))[0]
    #     suffix = f"{rank:05d}_{base}"
    #     Image.fromarray(render['real']).save(os.path.join(save_dir, f"{suffix}{i}_a.jpg"))
    #     Image.fromarray(pred_bin).save(os.path.join(save_dir, f"{suffix}{i}_pred_a.jpg"))
    #     Image.fromarray(entropy_color).save(os.path.join(save_dir, f"{suffix}{i}_en_a.jpg"))

    
    if rank>0:
        break

In [ ]:
# import os, random
# import numpy as np
# from PIL import Image
# import torch
# import torch.nn.functional as F
# import matplotlib.cm as cm

# save_dir = "entropy_sorted"
# os.makedirs(save_dir, exist_ok=True)

# def shift_and_resize_prompt(prompt, crop_x, crop_y, crop_size, orig_w, orig_h):
#     """
#     Shift prompt points based on crop (x, y), 
#     keep points inside the crop, and resize them to (orig_w, orig_h).
#     """
#     points = prompt[0][0]  # tensor shape: [N, 2]
#     labels = prompt[0][1]  # tensor shape: [N] or [N, 1]

#     # Shift coordinates by crop offset
#     points[:, 0] -= crop_x
#     points[:, 1] -= crop_y

#     # Keep points inside the crop region
#     mask_inside = (
#         (points[:, 0] >= 0) & (points[:, 1] >= 0) &
#         (points[:, 0] < crop_size) & (points[:, 1] < crop_size)
#     )
#     points = points[mask_inside]
#     labels = labels[mask_inside]

#     if len(points) == 0:
#         # If no valid points remain, return dummy far-out point
#         points = torch.tensor([[orig_w + 1000, orig_h + 1000]], device=prompt[0].device)
#         labels = torch.tensor([0], device=prompt[0].device)

#     # Rescale points from crop_size → original image size (since resized)
#     scale_x = orig_w / crop_size
#     scale_y = orig_h / crop_size
#     points[:, 0] *= scale_x
#     points[:, 1] *= scale_y

#     return (points, labels)

# patch_size = 256  # size of random patch before resize
# num_patches = 5   # number of random patches per image

# def get_random_patch_coords(h, w, patch_size):
#     y = random.randint(0, max(0, h - patch_size))
#     x = random.randint(0, max(0, w - patch_size))
#     return y, x

# for rank, (entropy_scalar, img_path, render) in enumerate(collected, start=1):
#     img = render['real']
#     H, W = img.shape[:2]

#     img_tensor_full = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0
#     img_tensor_full = img_tensor_full.unsqueeze(0).to(fabric.device)

#     prompt_b = render['prompt']

#     pseudo_accum = torch.zeros((1, H, W), dtype=torch.float32, device=fabric.device)
#     count = 0

#     # === Random patch extraction + resized inference ===
#     for _ in range(num_patches):
#         y, x = get_random_patch_coords(H, W, patch_size)
#         patch = img[y:y+patch_size, x:x+patch_size, :]

#         # --- Resize patch to full original image size before model inference ---
#         patch_resized = np.array(Image.fromarray(patch).resize((W, H), Image.BILINEAR))
#         patch_tensor = torch.from_numpy(patch_resized).permute(2, 0, 1).float() / 255.0
#         patch_tensor = patch_tensor.unsqueeze(0).to(fabric.device)

#         ref_prompt = shift_and_resize_prompt(prompt_b, x, y, patch_size, W, H)  # same or adaptive prompt

#         with torch.no_grad():
#             _, masks_pred, _, _ = model(patch_tensor, ref_prompt)
#             p = masks_pred[0].clamp(1e-6, 1 - 1e-6)
#             p = p.max(dim=0)[0]
#             pseudo_accum += p.unsqueeze(0)
#             count += 1

#     # === Average pseudo-label across all resized patch predictions ===
#     pseudo_mask = pseudo_accum / (count + 1e-6)

#     # === Entropy computation ===
#     entropy_map = - (pseudo_mask * torch.log(pseudo_mask + 1e-6) +
#                      (1 - pseudo_mask) * torch.log(1 - pseudo_mask + 1e-6))
#     entropy_np = entropy_map.squeeze(0).cpu().numpy()
#     entropy_norm = (entropy_np - entropy_np.min()) / (entropy_np.max() - entropy_np.min() + 1e-6)
#     cmap = cm.get_cmap("viridis")
#     entropy_color = (cmap(entropy_norm)[:, :, :3] * 255).astype(np.uint8)

#     # === Binarized pseudo mask ===
#     pseudo_bin = (pseudo_mask > 0.5).to(torch.uint8).squeeze(0).cpu().numpy() * 255

#     base = os.path.splitext(os.path.basename(img_path))[0]
#     suffix = f"{rank:05d}_{base}"

#     Image.fromarray(img).save(os.path.join(save_dir, f"{suffix}.jpg"))
#     Image.fromarray(pseudo_bin).save(os.path.join(save_dir, f"{suffix}_pseudo.jpg"))
#     Image.fromarray(entropy_color).save(os.path.join(save_dir, f"{suffix}_entropy.jpg"))

#     # === Optional: negative prompt recalibration ===
#     # new_prom = neg_prompt_calibration(cfg, [entropy_map], prompt_b)

#     if rank > 0:
#         break
